In [ ]:
#non questo
%%cython
import numpy as np
cimport numpy as np  # Import C numpy for typed memoryviews
import time
np.random.seed(42)
cdef double sign(double x):
          if x > 0:
              return 1.0
          elif x < 0:
              return -1.0
          else:
              return 0.0
def do_some_training_EN(URM_train,
                     double initial_learning_rate,
                     double regularization_2,
                     double l1_ratio,  # ElasticNet's L1 ratio
                     double decay_rate,
                     int num_iterations,
                     np.float64_t[:, :] existing_item_item_S):

    # Ensure URM_train is in CSR format for efficient row slicing
    URM_train_csr = URM_train.tocsr()
    URM_train_coo = URM_train.tocoo()
    n_items = URM_train.shape[1]

    cdef np.float64_t[:, :] item_item_S
    # Use the provided item-item similarity matrix if available; otherwise, initialize with zeros
    if existing_item_item_S is not None:
        item_item_S = existing_item_item_S
    else:
        item_item_S = np.zeros((n_items, n_items), dtype=np.float64)

    # Initial learning rate and regularization
    cdef np.float64_t learning_rate = initial_learning_rate
    cdef np.float64_t regularization_2_val = regularization_2
    cdef np.float64_t l1_ratio_val = l1_ratio
    cdef np.float64_t loss = 0.0
    cdef long start_time = time.time()
    cdef np.float64_t true_rating, predicted_rating, prediction_error, profile_rating
    cdef np.int32_t[:] items_in_user_profile
    cdef np.float64_t[:] ratings_in_user_profile
    cdef int index, sample_num, user_id, item_id, profile_item_id, sample_index

    cdef np.float64_t decay_rate_val = decay_rate
    # Initialize a list to store loss values
    cdef list loss_history = []

    # Early stopping variables
    cdef int patience_counter = 0
    cdef double last_loss = np.inf
    cdef int patience = 20
    cdef double min_delta = 1e-5
    cdef int warm_restart = 0

    cdef double delta=0

    cdef int[:] random_indices = np.random.randint(0, URM_train_coo.nnz, size=num_iterations).astype(np.int32)
    # Memoryviews for efficient access
    cdef int[:] indices = URM_train_csr.indices
    cdef int[:] indptr = URM_train_csr.indptr
    cdef double[:] data = URM_train_csr.data
    cdef int[:] coo_row = URM_train_coo.row
    cdef int[:] coo_col = URM_train_coo.col
    cdef double[:] coo_data = URM_train_coo.data
    # Use `indices`, `indptr`, and `data` within the loop

    for sample_num in range(num_iterations):
        sample_index = random_indices[sample_num]
        user_id = coo_row[sample_index]
        item_id = coo_col[sample_index]
        true_rating = coo_data[sample_index]

        # Compute prediction
        items_in_user_profile = indices[indptr[user_id]:indptr[user_id+1]]
        ratings_in_user_profile = data[indptr[user_id]:indptr[user_id+1]]
        predicted_rating = 0.0

        for index in range(len(items_in_user_profile)):
            profile_item_id = items_in_user_profile[index]
            profile_rating = ratings_in_user_profile[index]
            predicted_rating += profile_rating * item_item_S[profile_item_id, item_id]

        # Compute prediction error
        prediction_error = true_rating - predicted_rating
        loss += prediction_error ** 2

        # Update model (ElasticNet: L2 and L1 penalties)
        for index in range(len(items_in_user_profile)):
            profile_item_id = items_in_user_profile[index]
            profile_rating = ratings_in_user_profile[index]
            delta = learning_rate * (
              prediction_error * profile_rating
              - (1 - l1_ratio_val) * regularization_2_val * item_item_S[profile_item_id, item_id]
              - l1_ratio_val * sign(item_item_S[profile_item_id, item_id])
            )
            item_item_S[profile_item_id, item_id] += delta
            item_item_S[item_id, profile_item_id] += delta

        # Print some stats
        if sample_num % 5000 == 0:
            learning_rate *= decay_rate_val
            current_loss = loss / sample_num
            loss_history.append(current_loss)
            elapsed_time = time.time() - start_time
            samples_per_second = sample_num / elapsed_time
            print(
                "Iteration {} in {:.2f} seconds, loss is {:.4f}. Samples per second {:.2f}".format(
                    sample_num, elapsed_time, current_loss, samples_per_second
                )
            )
            # Early stopping check
            if abs(last_loss - current_loss) < min_delta or current_loss > 1:
                patience_counter += 1
                if patience_counter >= patience:
                    if warm_restart == 0:
                        learning_rate = initial_learning_rate
                        warm_restart += 1
                        patience_counter = 0
                        print("warm restart")
                    else:
                        print("Early stopping at iteration {}. Loss has not improved significantly for {} iterations, or has stayed above 1 too much".format(sample_num, patience * 5000))
                        break
            else:
                patience_counter = 0  # Reset patience counter if loss improves
            last_loss = current_loss
            # Imposta la diagonale a zero se necessario, ogni 5000
            for i in range(n_items):
                item_item_S[i, i] = 0.0
    item_item_S = (item_item_S + item_item_S.T) / 2
    return loss, samples_per_second, item_item_S, loss_history
